In [1]:
from time import time
import random
import numpy as np

### Evaluation function 
#### !! (added one more input element -> book_scores)

In [2]:
def evaluate_solution(solution, books_scores):
    books_scanned = []
    points = 0
    number_of_libraries = solution[0]
    #print("Num of libraries: ", number_of_libraries)
    for i in range(1, number_of_libraries+1):
        number_of_books = solution[i][1]
        #print("Num of books in libr: ",i, ":", number_of_books)
        for j in range(number_of_books):
            book_to_add = solution[i][2][j]
            if book_to_add not in books_scanned:
                books_scanned.append(book_to_add)
                #print("add: ", book_to_add)
    #print("Books scanned: ", books_scanned)
    #print("Books scores: ", books_scores)
    solution_score = 0
    for i in books_scanned:
        solution_score += books_scores[i]
        
    return solution_score

### Initial population
#### (changed 'from random import choice' to 'import random' and changed that in your code)

In [3]:
def get_random_solution(libraries_number, days_number, libraries, books_scores):
    #generate solution
    solution_indices = []
    time_pointer = 0
    while True:
        random_index = random.choice(list(range(libraries_number)))
        random_library = libraries[random_index]
        if time_pointer + random_library[0][1] > days_number:
            break                                 
        if random_index not in solution_indices:
            solution_indices.append(random_index)
            time_pointer += random_library[0][1]
    #create complete solution
    time_pointer = days_number
    used_books = []
    complete_solution = [len(solution_indices)]
    for i in solution_indices:
        time_pointer -= libraries[i][0][1]
        books_per_day = libraries[i][0][2]
        available_books = list(libraries[i][1])
        particular_solution = [i, 0, []]
        available_books.sort(key = (lambda x: books_scores[x]))
        for x in available_books:
            if len(particular_solution[2]) > time_pointer * books_per_day:
                break
            if x not in used_books:
                particular_solution[2].append(x)
                particular_solution[1] += 1
                used_books.append(x)
        particular_solution[2] = tuple(particular_solution[2])
        complete_solution.append(tuple(particular_solution))
    return tuple(complete_solution)
        
    
def get_initial_population(population_size, libraries_number, days_number, libraries, books):
    population = []
    for _ in range(population_size):
        population.append(get_random_solution(libraries_number, days_number, libraries, books))
    return population    

### Mutation

In [4]:
def mutate(solution):
    pass

def do_mutations(population):
    mutated_solutions = []
    for solution in population:
        mutated_solutions.append(mutate(solution))
    return mutated_solutions

## Crossover
#### !!! (Added an input param to get_children -> so also to get_offspring, did a function is_valid to be easier)

In [5]:
def is_valid_child(child, days_number):
    sign_in_time = 0
    for i in range(1, child[0]):
        lib = child[i][0]
        sign_in_time += libraries[lib][0][1]
        while(sign_in_time > days_number):
            child = (child[0]-1,)+child[1:child[0]-1]
            is_valid_child(child)
    return child

In [15]:
def get_children(days_number, solution_a, solution_b, libraries):
    length_a = solution_a[0]
    length_b = solution_b[0]
    xpoint_a = int((length_a+1)/2)
    xpoint_b = int((length_b+1)/2)
    #print(xpoint_a, xpoint_b)
    
    chunk_a1 = solution_a[1:xpoint_a]
    chunk_a2 = solution_a[xpoint_a:]
    chunk_b1 = solution_b[1:xpoint_b]
    chunk_b2 = solution_b[xpoint_b:]
    a = (len(chunk_a1+chunk_b2))
    b = (len(chunk_b1+chunk_a2))
    children_a = (a,)+chunk_a1+chunk_b2
    children_b = (b,)+chunk_b1+chunk_a2
    #print(children_a)
    #print(children_b)

    is_valid_child(children_a, days_number)
    is_valid_child(children_b, days_number)
    
    return children_a , children_b

def get_offspring(days_number, population, libraries):
    # let the population_size be EVEN
    offspring = tuple()
    for i in range (int(population_size/2)):
        offspring += get_children(days_number, population[i], population[i+1], libraries)
        
    return offspring

# Main

<font color="blue">__Specify how long program is able to run here (in seconds)__</blue>

In [16]:
max_duration = 300
epochs_duration = []
epochs_duration.append(time())
epochs_duration.append(time())

<font color="blue">__Specify population size here__</font>

In [17]:
population_size = 10

### Read data

<font color=blue>__Specify path to input file here__</font>

In [18]:
file = open("instances/a_example.txt", "r")

Load data from source file to the program

In [19]:
books_number, libraries_number, days_number = [int(x) for x in file.readline().split()]
books_scores = tuple([int(x) for x in file.readline().split()])
libraries = [None] * libraries_number
for i in range(libraries_number):
    libraries[i] = (tuple([int(x) for x in file.readline().split()]), tuple([int(x) for x in file.readline().split()]))
libraries = tuple(libraries) 

Get initial population

In [20]:
population = get_initial_population(population_size, libraries_number, days_number, libraries, books_scores)    

main loop - epochs <br>


In [21]:
while epochs_duration[0] + max_duration >= time() + (epochs_duration[-1] - epochs_duration[-2]) * 2:
    offspring = get_offspring(days_number, population, libraries)
    mutations = do_mutations(population)
    population.extend(offspring)
    population.extend(mutations)
    population.sort()
    population = population[:population_size]
    epochs_duration.append(time())

TypeError: '<' not supported between instances of 'NoneType' and 'tuple'

<hr>

### Useful add-ons

In [22]:
def print_input():
    #meta data
    print('Meta data:')
    print('\tBooks     -> ' + str(books_number))
    print('\tLibraries -> ' + str(libraries_number))
    print('\tDays      -> ' + str(days_number))
    
    #books scores
    print('Books\' scores:\n\t' + str(books_scores))

    #libraries
    print('Libraries:')
    for i, lib in zip(range(len(libraries)), libraries):
        print('\tLibrary no. ' + str(i) + ':')
        print('\t\tNumber of books   -> ' + str(lib[0][0]))
        print('\t\tDays to sign up   -> ' + str(lib[0][1]))
        print('\t\tBook ship per day -> ' + str(lib[0][2]))
        print('\t\tAvailiable books:')
        print('\t\t\t' + str(lib[1]) + '\n')
    
print_input()

Meta data:
	Books     -> 6
	Libraries -> 2
	Days      -> 7
Books' scores:
	(1, 2, 3, 6, 5, 4)
Libraries:
	Library no. 0:
		Number of books   -> 5
		Days to sign up   -> 2
		Book ship per day -> 2
		Availiable books:
			(0, 1, 2, 3, 4)

	Library no. 1:
		Number of books   -> 4
		Days to sign up   -> 3
		Book ship per day -> 1
		Availiable books:
			(0, 2, 3, 5)

